In [1]:
import time as time
import os
import sys
import io

import tracemalloc
import numpy as np
import pandas as pd
import sklearn as skl

from sklearn.linear_model import lasso_path
from pysnptools.snpreader import Bed

# import names_in_pipeline as nip
# import pipeline_utilities as pu
import scipy as sp

#For plotting
import matplotlib.pyplot as plt


In [2]:
work_path = '/mnt/home/rabentim/simulations'
gene_path = f'{work_path}/simulated' # '/full/path/to/plink/files'
bim = pd.read_csv(f'{gene_path}.bim',header=None,sep='\t')
fam = pd.read_csv(f'{gene_path}.fam',header=None,sep='\s+')

In [3]:
#specify the number of features per block to be loaded
nsnps_per = 50
#specify the number of blocks (in this case the number of chromosomes)
nblocks=22
#specify the number of samples
nsample = 8000

#list of snps to use. nsnps_per are randomly sampled from each block. This must be edited if you want to work on non-chromosome blocks
all_snps = [bim[bim[0]==i][1][0:nsnps_per] for i in np.sort(np.random.choice(range(1,23),nblocks,replace=False))]
#fam file of just samples
sub_fam = fam.sample(nsample)
sub_fam = sub_fam.sort_values(by=1)
#snpreader Bed file
bed_data = Bed(gene_path,count_A1=False)
print(f'using {nsample} samples and {nsnps_per*nblocks} features')

using 8000 samples and 1100 features


# global lasso

## loading genetic data

Loading each block and combining them

In [55]:
tracemalloc.start()
tracemalloc.reset_peak()
t = time.time()
iids = sub_fam[[0,1]].values.astype(str)
sample_idx = bed_data.iid_to_index(iids)
for i in range(0,nblocks):
    top_snps_idx = bed_data.sid_to_index(all_snps[i])
    sub_snpreader = bed_data[sample_idx, top_snps_idx]
    snpdata = sub_snpreader.read()
    tmpG=snpdata.val
    if i == 0:
        subG=tmpG
    else:
        subG = np.hstack((subG,tmpG))
ellapse = time.time() - t
print(f'{ellapse} seconds')
malloc = tracemalloc.get_traced_memory()
malloc = tuple(it/1000000000 for it in malloc)
print(f'tracemalloc: {malloc} GB of RAM')
print(f'final shape: {np.shape(subG)}')
global_malloc = malloc
tracemalloc.stop()

0.6166880130767822 seconds
tracemalloc: (0.074572756, 0.141771956) GB of RAM
final shape: (8000, 1100)


Or loading all snps at once

In [56]:
# tracemalloc.start()
# tracemalloc.reset_peak()
# t = time.time()
# iids = sub_fam[[0,1]].values.astype(str)
# sample_idx = bed_data.iid_to_index(iids)
# top_snps_idx = bed_data.sid_to_index(np.hstack(all_snps))
# sub_snpreader = bed_data[sample_idx, top_snps_idx]
# snpdata = sub_snpreader.read()
# subG=snpdata.val
# ellapse = time.time() - t
# print(f'{ellapse} seconds')
# malloc = tracemalloc.get_traced_memory()
# malloc = tuple(it/1000000000 for it in malloc)
# print(f'tracemalloc: {malloc} GB of RAM')
# print(f'final shape: {np.shape(subG)}')
# global_malloc = malloc
# tracemalloc.stop()

In [8]:
#separate genotype matrix into training, validation, and testing sets: train 80%, val 10%, test 10% 
trainG = subG[0:np.floor(.8*nsample).astype(int)]
valG = subG[np.floor(.8*nsample).astype(int):np.floor(.9*nsample).astype(int)]
testG = subG[np.floor(.9*nsample).astype(int):]

In [9]:
#standardize the genetic matrices according to the training set
center = np.zeros(trainG.shape[1])
spread = np.zeros(trainG.shape[1])
for col in range(0,subG.shape[1]):
    center[col] = np.nanmean(trainG[:,col])
    spread[col] = np.nanstd(trainG[:,col])

missing = np.argwhere(np.isnan(subG))
for row in range(0,missing.shape[0]):
    ind1 = missing[row,0]
    ind2 = missing[row,1]
    subG[ind1,ind2] = center[ind2]

for col in range(0,subG.shape[1]):
    val = spread[col]
    if spread[col] == 0.0:
        val = 1.0
    subG[:,col] = (subG[:,col] - center[col])/val

trainG = subG[0:np.floor(.8*nsample).astype(int)]
valG = subG[np.floor(.8*nsample).astype(int):np.floor(.9*nsample).astype(int)]
testG = subG[np.floor(.9*nsample).astype(int):]

Features (SNPs) should be roughly indpendent between blocks for the blockLASSO to work well. This can be visually seen by uncommenting the following code and plotting the correlation structure. Warning this can be slow for large matrices.

In [10]:
# feature = np.random.randint(0,nsnps_per*nblocks)
# df=pd.DataFrame(subG)
# plt.scatter(np.linspace(1,nsnps_per*nblocks,nsnps_per*nblocks),df.corr()[feature])
# for i in range(1,nblocks):
#     plt.axvline(i*nsnps_per,-0.2,1,color='red',ls='--')

## generate genetic model and train global LASSO

we will assume a simple linear model with a noise term (enviornmental, covariate, etc.) and no explicit covariates

In [11]:
#define number of participating features and assign them effect weights in [0,1). 
# err_rel_size defines the error scale in terms of the standard deviation of the true weight effects
ntrue_weights = 300
err_rel_size = 5
tmp = np.zeros(nblocks*nsnps_per)
weights = np.random.rand(ntrue_weights)
weight_locations = np.sort(np.random.randint(0,nblocks*nsnps_per,ntrue_weights))
tmp[weight_locations] = weights
weights = tmp
gene_effects = np.dot(subG,weights)
error_effects = err_rel_size*np.std(gene_effects)* np.random.rand(nsample)

#define true phenotype and split into various sets
true_y = gene_effects + error_effects
train_y = true_y[0:np.floor(.8*nsample).astype(int)]
val_y = true_y[np.floor(.8*nsample).astype(int):np.floor(.9*nsample).astype(int)]
test_y = true_y[np.floor(.9*nsample).astype(int):]

#standardize training set for algorithm training
ymu = np.mean(train_y)
ysig = np.std(train_y)
norm_train_y = (train_y-ymu)/ysig

In [12]:
#run global lasso
tracemalloc.start()
tracemalloc.reset_peak()
t = time.time()
path = skl.linear_model.lasso_path(trainG,norm_train_y,n_alphas=50,eps=.01,max_iter=1500)
ellapse = time.time() - t
print(f'{ellapse} seconds')
malloc = tracemalloc.get_traced_memory()
malloc = tuple(it/1000000000 for it in malloc)
print(f'tracemalloc: {malloc} GB of RAM')
global_lasso_malloc = malloc
tracemalloc.stop()

1.412977933883667 seconds
tracemalloc: (0.000453321, 0.066655798) GB of RAM


In [13]:
# correlation in validation set
val_mets =np.array([np.corrcoef(val_y,np.dot(valG,path[1])[:,i])[0][1] for i in range(0,50)])
max_ind = np.nanargmax(val_mets)
# correlation in testing set
global_met = np.corrcoef(test_y,np.dot(testG,path[1][:,max_ind]))[0][1]
#True correlation
true_met = np.corrcoef(gene_effects,true_y)[0][1]

/mnt/home/rabentim/.local/lib/python3.11/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/mnt/home/rabentim/.local/lib/python3.11/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


# blockLASSO

## loading genetic data

In [67]:
tracemalloc.start()
tracemalloc.reset_peak()
t = time.time()
iids = sub_fam[[0,1]].values.astype(str)
sample_idx = bed_data.iid_to_index(iids)
for i in range(0,nblocks):
    top_snps_idx = bed_data.sid_to_index(all_snps[i])
    sub_snpreader = bed_data[sample_idx, top_snps_idx]
    snpdata = sub_snpreader.read()
    tmpG=snpdata.val
    if i ==0:
        malloc = tracemalloc.get_traced_memory()
        malloc = tuple(it/1000000000 for it in malloc)
        print(f'single (first) block tracemalloc: {malloc} GB of RAM')
        single_block_malloc = malloc

        subG=tmpG
    else:
        subG = np.hstack((subG,tmpG))
ellapse = time.time() - t
print(f'{ellapse} seconds')
malloc = tracemalloc.get_traced_memory()
malloc = tuple(it/1000000000 for it in malloc)
print(f'total tracemalloc: {malloc} GB of RAM')
print(f'final shape: {np.shape(subG)}')
block_malloc = malloc
tracemalloc.stop()

single (first) block tracemalloc: (0.004171469, 0.004364953) GB of RAM
0.6138358116149902 seconds
total tracemalloc: (0.074572813, 0.141772802) GB of RAM
final shape: (8000, 1100)


In [16]:
# train 80%, val 10%, test 10% 
trainG = subG[0:np.floor(.8*nsample).astype(int)]
valG = subG[np.floor(.8*nsample).astype(int):np.floor(.9*nsample).astype(int)]
testG = subG[np.floor(.9*nsample).astype(int):]

In [17]:
center = np.zeros(trainG.shape[1])
spread = np.zeros(trainG.shape[1])
for col in range(0,subG.shape[1]):
    center[col] = np.nanmean(trainG[:,col])
    spread[col] = np.nanstd(trainG[:,col])

missing = np.argwhere(np.isnan(subG))
for row in range(0,missing.shape[0]):
    ind1 = missing[row,0]
    ind2 = missing[row,1]
    subG[ind1,ind2] = center[ind2]

for col in range(0,subG.shape[1]):
    val = spread[col]
    if spread[col] == 0.0:
        val = 1.0
    subG[:,col] = (subG[:,col] - center[col])/val

trainG = subG[0:np.floor(.8*nsample).astype(int)]
valG = subG[np.floor(.8*nsample).astype(int):np.floor(.9*nsample).astype(int)]
testG = subG[np.floor(.9*nsample).astype(int):]

In [19]:
paths = []
block_lasso_mallocs = []
tracemalloc.start()
tracemalloc.reset_peak()
t = time.time()
for i in range(0,nblocks):
    tmp_path = skl.linear_model.lasso_path(trainG[:,i*nsnps_per:(i+1)*nsnps_per],
                                   norm_train_y,n_alphas=50,eps=.01,max_iter=1500)
    paths.append(tmp_path)
    ellapse = time.time() - t
    print(f'{ellapse} seconds')
    malloc = tracemalloc.get_traced_memory()
    block_lasso_mallocs.append(malloc)
    malloc = tuple(it/1000000000 for it in malloc)
    print(f'tracemalloc: {malloc} GB of RAM')
tracemalloc.stop()

0.060268402099609375 seconds
tracemalloc: (2.8792e-05, 0.002740521) GB of RAM
0.11486005783081055 seconds
tracemalloc: (5.0911e-05, 0.002764075) GB of RAM
0.1693711280822754 seconds
tracemalloc: (7.2671e-05, 0.002785924) GB of RAM
0.22194719314575195 seconds
tracemalloc: (9.4368e-05, 0.002807684) GB of RAM
0.2795383930206299 seconds
tracemalloc: (0.000115324, 0.002829257) GB of RAM
0.3318798542022705 seconds
tracemalloc: (0.000137303, 0.002850588) GB of RAM
0.3842184543609619 seconds
tracemalloc: (0.000158992, 0.002872245) GB of RAM
0.4354431629180908 seconds
tracemalloc: (0.000180617, 0.002893934) GB of RAM
0.48949146270751953 seconds
tracemalloc: (0.000201629, 0.002913766) GB of RAM
0.5421469211578369 seconds
tracemalloc: (0.000223582, 0.002936867) GB of RAM
0.5977065563201904 seconds
tracemalloc: (0.000245323, 0.002958576) GB of RAM
0.652940034866333 seconds
tracemalloc: (0.000266947, 0.002980265) GB of RAM
0.7058711051940918 seconds
tracemalloc: (0.000287956, 0.003000017) GB of RAM

In [20]:
#build validation correlations for each block
valGs =[]
for i in range(0,nblocks):
    valGs.append(valG[:,i*nsnps_per:(i+1)*nsnps_per])
vals_mets = np.array([np.array([np.corrcoef(val_y,np.dot(valGs[j],paths[j][1])[:,i])[0][1] for i in range(0,50)]) for j in range(0,nblocks)])
bests = np.array([np.nanargmax(vals_mets[i]) for i in range(0,nblocks)])

#perform linear regression on blocks
block_val_scores = np.array([np.dot(valGs[i],paths[i][1][:,bests[i]]) for i in range(0,nblocks)])
fit = skl.linear_model.LinearRegression().fit(block_val_scores.T,val_y)

#apply full model to testing set
testGs =[]
for i in range(0,nblocks):
    testGs.append(testG[:,i*nsnps_per:(i+1)*nsnps_per])
block_test_scores = np.array([np.dot(testGs[i],paths[i][1][:,bests[i]]) for i in range(0,nblocks)])

unweighted_met = np.corrcoef(np.sum(block_test_scores.T,axis=1),test_y)[0][1]
block_met = np.corrcoef(fit.predict(block_test_scores.T),test_y)[0][1]

/mnt/home/rabentim/.local/lib/python3.11/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/mnt/home/rabentim/.local/lib/python3.11/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


# final results

In [21]:
true_met, global_met, block_met

(0.5734302955302221, 0.5065586433125017, 0.482724768973531)

In [26]:
global_malloc

(0.139249189, 0.206448272)

In [27]:
global_lasso_malloc

(0.000453321, 0.066655798)

In [28]:
block_malloc

(0.074875703, 0.142074935)

In [31]:
block_lasso_avg_mallocs = np.mean(np.array(block_lasso_mallocs),axis=0).T/1000000000
block_lasso_avg_mallocs

array([0.00025607, 0.00296909])

# reading the script output

In [4]:
data = pd.read_csv(f'{work_path}/lasso_grid_data.txt')
data.columns

Index(['nsnps_per', 'nsample', 'sparsity', 'nblocks', 'global_malloc_current',
       'global_malloc_peak', 'global_lasso_malloc_current',
       'global_lasso_malloc_peak', 'true_met', 'global_met',
       'single_block_malloc_current', 'single_block_malloc_peak',
       'block_malloc_current', 'block_malloc_peak',
       'avg_block_lasso_malloc_current', 'avg_block_lasso_malloc_peak',
       'unweighted_block_met', 'block_met'],
      dtype='object')

In [12]:
# df = data[(data['nsnps_per']==100)&(data['sparsity']==.5)]
df = data[(data['nsample']==10000)&(data['sparsity']==.5)]

In [17]:
df[['nsnps_per','true_met','global_met','unweighted_block_met','block_met']]

,nsnps_per,true_met,global_met,unweighted_block_met,block_met
4,10,0.569898,0.549227,0.530837,0.540435
16,50,0.569183,0.516422,0.469073,0.505247
28,100,0.572298,0.570556,0.524793,0.544423
40,500,0.561924,0.493574,0.336756,0.436249


In [9]:
df[['global_malloc_current','global_lasso_malloc_current', 'single_block_malloc_current','block_malloc_current','avg_block_lasso_malloc_current']]

,global_malloc_current,global_lasso_malloc_current,single_block_malloc_current,block_malloc_current,avg_block_lasso_malloc_current
25,0.147007,0.000882,0.004611,0.092611,481080.590909
28,0.235608,0.000882,0.009211,0.185212,480401.727273
31,0.324207,0.000882,0.013811,0.277811,479239.227273
34,0.412807,0.000882,0.018411,0.370411,481662.818182


In [10]:
df['global_malloc_current']/df['single_block_malloc_current']

25    31.884247
28    25.579932
31    23.475163
34    22.422204
dtype: float64

In [11]:
data['global_malloc_current']/data['single_block_malloc_current']

0     67.500062
1     67.449281
2     67.448965
3     38.464084
4     38.464144
5     38.464076
6     28.784853
7     28.784830
8     28.784854
9     23.941948
10    23.941929
11    23.941931
12    39.486043
13    39.526632
14    39.486027
15    28.333335
16    28.333325
17    28.333345
18    24.609827
19    24.610237
20    24.609830
21    22.746959
22    22.746959
23    22.746959
24    31.884248
25    31.884247
26    31.884250
27    25.579864
28    25.579932
29    25.579871
30    23.475346
31    23.475163
32    23.475163
33    22.422200
34    22.422204
35    22.422204
36    24.207362
37    24.207363
38    24.207363
39    22.799424
40    22.799424
41    22.799425
42    22.329416
43    22.329416
44    22.329416
45    22.094280
46    22.094281
47    22.094280
dtype: float64